# 여자배구 데이터 분석, 경기 결과 예측 프로젝트

## 라이브러리 import

In [1]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
import findspark as fs

fs.init()

In [2]:
# spark 생성

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## 데이터 전처리 - 22/03/12

In [3]:
# 파일 불러오기
data = spark.read.csv("hdfs://localhost:9000/data/player.csv", header="true", inferSchema="true")

In [4]:
data.show()

+---+------+------------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|        team|pos| score|error|setcount|at_try|at_succ|at_err|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+------------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|  1|임명옥|한국도로공사| Li|1475.0|    3|     117|     1|      0|     1|      0|     123|      0|   677|    390|     16|    690|
|  2|신연경| IBK기업은행| Li|1158.6|    3|     112|     1|      0|     1|      0|      67|      0|   525|    229|     22|    591|
|  3|김연견|    현대건설| Li|1099.0|    4|     113|     0|      0|     0|      0|      89|      0|   477|    213|     26|    560|
|  4|오지영|    GS칼텍스| Li| 880.6|    2|      94|     1|      0|     1|      0|      48|      0|   541|    249|     36|    375|
|  5|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     1|      0|     1|      0|      69|      0|   568|    224|     42|    307

In [5]:
# 불러온 데이터 테이블로 만들기
data.createOrReplaceTempView("player")

In [8]:
# SQL문 활용해서 데이터 조회
player = spark.sql("select _c0,name,team,pos,score from player")
player.show()

+---+------+------------+---+------+
|_c0|  name|        team|pos| score|
+---+------+------------+---+------+
|  1|임명옥|한국도로공사| Li|1475.0|
|  2|신연경| IBK기업은행| Li|1158.6|
|  3|김연견|    현대건설| Li|1099.0|
|  4|오지영|    GS칼텍스| Li| 880.6|
|  5|문슬기|페퍼저축은행| Li| 841.8|
|  6|  노란| KGC인삼공사| Li| 819.8|
|  7|김해란|    흥국생명| Li| 596.4|
|  8|도수빈|    흥국생명| Li| 589.2|
|  9|김세인|페퍼저축은행| Li| 334.4|
| 10|채선아| KGC인삼공사| Li| 331.2|
| 11|한수진|    GS칼텍스| Li| 222.6|
| 12|한다혜|    GS칼텍스| Li|  86.2|
| 13|김수빈| IBK기업은행| Li|  83.6|
| 14|이영주|    현대건설| Li|  62.2|
| 15|김주하|    현대건설| Li|  59.8|
| 16|박상미|    흥국생명| Li|  42.2|
| 17|이은지|페퍼저축은행| Li|   7.8|
| 18|김주희|    GS칼텍스| Li|   3.0|
| 19|박혜미|한국도로공사| Li|   3.0|
| 20|차유정|    GS칼텍스| Li|   3.0|
+---+------+------------+---+------+
only showing top 20 rows



In [37]:
spark.sql("select distinct team from player").show()

+------------+
|        team|
+------------+
|    흥국생명|
|한국도로공사|
|    GS칼텍스|
|    현대건설|
|페퍼저축은행|
| KGC인삼공사|
| IBK기업은행|
+------------+



In [9]:
# 구단 별 테이블 만들기

hgun = spark.sql("""
          select *
          from player
          where team like '현대건설'
          """)
hgun.show(5)

+---+------+--------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|    team|pos| score|error|setcount|at_try|at_succ|at_err|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+--------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|  3|김연견|현대건설| Li|1099.0|    4|     113|     0|      0|     0|      0|      89|      0|   477|    213|     26|    560|
| 14|이영주|현대건설| Li|  62.2|    2|      21|     1|      0|     1|      0|       4|      0|    15|      6|      2|     28|
| 15|김주하|현대건설| Li|  59.8|    0|       5|     0|      0|     0|      0|       5|      0|    24|     11|      3|     34|
| 21|한미르|현대건설| Li|   1.4|    0|       1|     0|      0|     0|      0|       0|      0|     0|      0|      0|      0|
| 33|황민경|현대건설|  L|1815.0|   69|     112|   536|    171|    25|     21|      28|     14|   527|    186|     22|    428|
+---+------+--------+---+--

In [10]:
dogong = spark.sql("""
          select *
          from player
          where team like '한국도로공사'
          """)
dogong.show(5)

+---+------+------------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|        team|pos| score|error|setcount|at_try|at_succ|at_err|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+------------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|  1|임명옥|한국도로공사| Li|1475.0|    3|     117|     1|      0|     1|      0|     123|      0|   677|    390|     16|    690|
| 19|박혜미|한국도로공사| Li|   3.0|    0|       3|     0|      0|     0|      0|       0|      0|     0|      0|      0|      0|
| 22|김정아|한국도로공사| Li|   1.0|    0|       1|     0|      0|     0|      0|       0|      0|     0|      0|      0|      0|
| 30|박정아|한국도로공사|  L|2094.0|  113|     115|  1153|    393|    74|     13|      13|     34|   141|     26|     16|    256|
| 37|전새얀|한국도로공사|  L|1500.0|   49|     115|   510|    179|    26|      7|      14|     39|   556|    163|     28|    192|
+---+

In [11]:
gscal = spark.sql("""
          select *
          from player
          where team like 'GS칼텍스'
          """)
gscal.show(5)

+---+------+--------+---+-----+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|    team|pos|score|error|setcount|at_try|at_succ|at_err|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+--------+---+-----+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|  4|오지영|GS칼텍스| Li|880.6|    2|      94|     1|      0|     1|      0|      48|      0|   541|    249|     36|    375|
| 11|한수진|GS칼텍스| Li|222.6|    0|      59|     0|      0|     0|      0|      15|      0|     0|      0|      0|    133|
| 12|한다혜|GS칼텍스| Li| 86.2|    0|      15|     0|      0|     0|      0|       4|      0|    56|     29|      0|     24|
| 18|김주희|GS칼텍스| Li|  3.0|    0|       3|     0|      0|     0|      0|       0|      0|     0|      0|      0|      0|
| 20|차유정|GS칼텍스| Li|  3.0|    0|       3|     0|      0|     0|      0|       0|      0|     0|      0|      0|      0|
+---+------+--------+---+-----

In [12]:
insam = spark.sql("""
          select *
          from player
          where team like 'KGC인삼공사'
          """)
insam.show(5)

+---+------+-----------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|       team|pos| score|error|setcount|at_try|at_succ|at_err|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+-----------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|  6|  노란|KGC인삼공사| Li| 819.8|    5|      83|     3|      0|     3|      0|      54|      0|   357|    157|     12|    416|
| 10|채선아|KGC인삼공사| Li| 331.2|    1|      37|     0|      0|     0|      0|      25|      0|   183|     82|     10|    140|
| 23|서유경|KGC인삼공사| Li|   1.0|    0|       1|     0|      0|     0|      0|       0|      0|     0|      0|      0|      0|
| 26|이소영|KGC인삼공사|  L|2311.4|  102|     105|   873|    310|    51|     22|      32|     31|   616|    267|     23|    391|
| 38|박혜민|KGC인삼공사|  L|1440.6|   68|      85|   486|    173|    34|     11|      13|     16|   528|    182|     29|    251|
+-

In [13]:
ibk = spark.sql("""
          select *
          from player
          where team like 'IBK기업은행'
          """)
ibk.show(5)

+---+------+-----------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|       team|pos| score|error|setcount|at_try|at_succ|at_err|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+-----------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|  2|신연경|IBK기업은행| Li|1158.6|    3|     112|     1|      0|     1|      0|      67|      0|   525|    229|     22|    591|
| 13|김수빈|IBK기업은행| Li|  83.6|    0|      26|     0|      0|     0|      0|       2|      0|    15|      5|      1|     39|
| 24|양유경|IBK기업은행| Li|   1.0|    0|       1|     0|      0|     0|      0|       0|      0|     0|      0|      0|      0|
| 28|표승주|IBK기업은행|  L|2175.4|   57|     102|   775|    285|    29|     17|      24|     28|   755|    231|     30|    302|
| 35|김주향|IBK기업은행|  L|1704.4|   76|      91|   721|    238|    48|      8|      10|     20|   521|    163|     23|    267|
+--

In [14]:
hguk = spark.sql("""
          select *
          from player
          where team like '흥국생명'
          """)
hguk.show(5)

+---+------+--------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|    team|pos| score|error|setcount|at_try|at_succ|at_err|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+--------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|  7|김해란|흥국생명| Li| 596.4|    2|      58|     0|      0|     0|      0|      59|      0|   211|     82|     16|    335|
|  8|도수빈|흥국생명| Li| 589.2|    3|      60|     1|      0|     1|      0|      38|      0|   271|    116|     12|    294|
| 16|박상미|흥국생명| Li|  42.2|    0|      16|     0|      0|     0|      0|       0|      0|     3|      1|      1|     23|
| 25|현무린|흥국생명| Li|   1.0|    0|       1|     0|      0|     0|      0|       0|      0|     0|      0|      0|      0|
| 27|김미연|흥국생명|  L|2238.2|  102|     111|   751|    250|    46|     27|      35|     21|   790|    251|     49|    406|
+---+------+--------+---+--

In [15]:
pepper = spark.sql("""
          select *
          from player
          where team like '페퍼저축은행'
          """)
pepper.show(5)

+---+------+------------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|        team|pos| score|error|setcount|at_try|at_succ|at_err|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+------------+---+------+-----+--------+------+-------+------+-------+--------+-------+------+-------+-------+-------+
|  5|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     1|      0|     1|      0|      69|      0|   568|    224|     42|    307|
|  9|김세인|페퍼저축은행| Li| 334.4|    0|      84|     0|      0|     0|      0|      19|      0|    36|     11|      0|    185|
| 17|이은지|페퍼저축은행| Li|   7.8|    0|       4|     0|      0|     0|      0|       0|      0|     0|      0|      0|      2|
| 32|이한비|페퍼저축은행|  L|1930.4|   91|     105|   791|    238|    40|     11|      21|     13|   695|    237|     33|    334|
| 34|박경현|페퍼저축은행|  L|1714.8|   94|      96|   610|    217|    30|     15|      22|     10|   741|    205|     41|    275|
+---+

#### 공성, 공효, 리효, 세트당 서브/세트/블로킹/디그 계산

공격 성공률(%) : 성공/시도 <br>
공격 효율(%) : (성공-범실)/시도  <br>
리시브 효율(%) : (정확-실패)/시도 --- "rs_rate":round((rscorr-rsfail)/rstry*100,2)  <br>
세트당 ~ (개) : 성공/시도

In [18]:
# 경기 지표 계산한 테이블 만들기

player_rate = spark.sql("""
          select name,team,pos,score,error,setcount,
              round((at_succ/at_try*100),2) as at_srate,round(((at_succ-at_err)/at_try*100),2) as at_rate,
              round(sv_succ/setcount,2) as sv, round(set_succ/setcount,2) as set,round(bl_succ/setcount,2) as bl,
              round(((rs_corr-rs_fail)/rs_try*100),2) as rs_rate,round(dg_succ/setcount,2) as dg
          from player
          """)

player_rate.show()

+------+------------+---+------+-----+--------+--------+-------+---+----+---+-------+----+
|  name|        team|pos| score|error|setcount|at_srate|at_rate| sv| set| bl|rs_rate|  dg|
+------+------------+---+------+-----+--------+--------+-------+---+----+---+-------+----+
|임명옥|한국도로공사| Li|1475.0|    3|     117|     0.0| -100.0|0.0|1.05|0.0|  55.24| 5.9|
|신연경| IBK기업은행| Li|1158.6|    3|     112|     0.0| -100.0|0.0| 0.6|0.0|  39.43|5.28|
|김연견|    현대건설| Li|1099.0|    4|     113|    null|   null|0.0|0.79|0.0|   39.2|4.96|
|오지영|    GS칼텍스| Li| 880.6|    2|      94|     0.0| -100.0|0.0|0.51|0.0|  39.37|3.99|
|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     0.0| -100.0|0.0| 0.7|0.0|  32.04| 3.1|
|  노란| KGC인삼공사| Li| 819.8|    5|      83|     0.0| -100.0|0.0|0.65|0.0|  40.62|5.01|
|김해란|    흥국생명| Li| 596.4|    2|      58|    null|   null|0.0|1.02|0.0|  31.28|5.78|
|도수빈|    흥국생명| Li| 589.2|    3|      60|     0.0| -100.0|0.0|0.63|0.0|  38.38| 4.9|
|김세인|페퍼저축은행| Li| 334.4|    0|      84|    null|   null|0.

In [33]:
# null 값, 의미없는 값 0으로 변경

player_rate2 = spark.sql("""
          select name,team,pos,score,error,setcount,
          case when at_srate is null then 0.0 else at_srate end as at_srate,
          case when at_rate is null or at_rate = -100.0 then 0.0 else at_rate end as at_rate,
          sv,set,bl,
          case when rs_rate is null then 0.0 else rs_rate end as rs_rate,
          dg
          from player_rate
          """)

player_rate2.show()
player_rate2.createOrReplaceTempView("player_rate2")

+------+------------+---+------+-----+--------+--------+-------+---+----+---+-------+----+
|  name|        team|pos| score|error|setcount|at_srate|at_rate| sv| set| bl|rs_rate|  dg|
+------+------------+---+------+-----+--------+--------+-------+---+----+---+-------+----+
|임명옥|한국도로공사| Li|1475.0|    3|     117|     0.0|    0.0|0.0|1.05|0.0|  55.24| 5.9|
|신연경| IBK기업은행| Li|1158.6|    3|     112|     0.0|    0.0|0.0| 0.6|0.0|  39.43|5.28|
|김연견|    현대건설| Li|1099.0|    4|     113|     0.0|    0.0|0.0|0.79|0.0|   39.2|4.96|
|오지영|    GS칼텍스| Li| 880.6|    2|      94|     0.0|    0.0|0.0|0.51|0.0|  39.37|3.99|
|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     0.0|    0.0|0.0| 0.7|0.0|  32.04| 3.1|
|  노란| KGC인삼공사| Li| 819.8|    5|      83|     0.0|    0.0|0.0|0.65|0.0|  40.62|5.01|
|김해란|    흥국생명| Li| 596.4|    2|      58|     0.0|    0.0|0.0|1.02|0.0|  31.28|5.78|
|도수빈|    흥국생명| Li| 589.2|    3|      60|     0.0|    0.0|0.0|0.63|0.0|  38.38| 4.9|
|김세인|페퍼저축은행| Li| 334.4|    0|      84|     0.0|    0.0|0.

In [34]:
player_rate2.createOrReplaceTempView("player_rate")

In [35]:
# 경기 지표 계산한 테이블 구단 별로 나누기

hgun_rate = spark.sql("""
          select *
          from player_rate
          where team like '현대건설'
          """)
hgun_rate.show(5)

+------+--------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|  name|    team|pos| score|error|setcount|at_srate|at_rate|  sv| set|  bl|rs_rate|  dg|
+------+--------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|김연견|현대건설| Li|1099.0|    4|     113|     0.0|    0.0| 0.0|0.79| 0.0|   39.2|4.96|
|이영주|현대건설| Li|  62.2|    2|      21|     0.0|    0.0| 0.0|0.19| 0.0|  26.67|1.33|
|김주하|현대건설| Li|  59.8|    0|       5|     0.0|    0.0| 0.0| 1.0| 0.0|  33.33| 6.8|
|한미르|현대건설| Li|   1.4|    0|       1|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0| 0.0|
|황민경|현대건설|  L|1815.0|   69|     112|    31.9|  27.24|0.19|0.25|0.13|  31.12|3.82|
+------+--------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
only showing top 5 rows



In [36]:
dogong_rate = spark.sql("""
          select *
          from player_rate
          where team like '한국도로공사'
          """)
dogong_rate.show(5)

+------+------------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|  name|        team|pos| score|error|setcount|at_srate|at_rate|  sv| set|  bl|rs_rate|  dg|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|임명옥|한국도로공사| Li|1475.0|    3|     117|     0.0|    0.0| 0.0|1.05| 0.0|  55.24| 5.9|
|박혜미|한국도로공사| Li|   3.0|    0|       3|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0| 0.0|
|김정아|한국도로공사| Li|   1.0|    0|       1|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0| 0.0|
|박정아|한국도로공사|  L|2094.0|  113|     115|   34.08|  27.67|0.11|0.11| 0.3|   7.09|2.23|
|전새얀|한국도로공사|  L|1500.0|   49|     115|    35.1|   30.0|0.06|0.12|0.34|  24.28|1.67|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
only showing top 5 rows



In [37]:
gscal_rate = spark.sql("""
          select *
          from player_rate
          where team like 'GS칼텍스'
          """)
gscal_rate.show(5)

+------+--------+---+-----+-----+--------+--------+-------+---+----+---+-------+----+
|  name|    team|pos|score|error|setcount|at_srate|at_rate| sv| set| bl|rs_rate|  dg|
+------+--------+---+-----+-----+--------+--------+-------+---+----+---+-------+----+
|오지영|GS칼텍스| Li|880.6|    2|      94|     0.0|    0.0|0.0|0.51|0.0|  39.37|3.99|
|한수진|GS칼텍스| Li|222.6|    0|      59|     0.0|    0.0|0.0|0.25|0.0|    0.0|2.25|
|한다혜|GS칼텍스| Li| 86.2|    0|      15|     0.0|    0.0|0.0|0.27|0.0|  51.79| 1.6|
|김주희|GS칼텍스| Li|  3.0|    0|       3|     0.0|    0.0|0.0| 0.0|0.0|    0.0| 0.0|
|차유정|GS칼텍스| Li|  3.0|    0|       3|     0.0|    0.0|0.0| 0.0|0.0|    0.0| 0.0|
+------+--------+---+-----+-----+--------+--------+-------+---+----+---+-------+----+
only showing top 5 rows



In [38]:
insam_rate = spark.sql("""
          select *
          from player_rate
          where team like 'KGC인삼공사'
          """)
insam_rate.show(5)

+------+-----------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|  name|       team|pos| score|error|setcount|at_srate|at_rate|  sv| set|  bl|rs_rate|  dg|
+------+-----------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|  노란|KGC인삼공사| Li| 819.8|    5|      83|     0.0|    0.0| 0.0|0.65| 0.0|  40.62|5.01|
|채선아|KGC인삼공사| Li| 331.2|    1|      37|     0.0|    0.0| 0.0|0.68| 0.0|  39.34|3.78|
|서유경|KGC인삼공사| Li|   1.0|    0|       1|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0| 0.0|
|이소영|KGC인삼공사|  L|2311.4|  102|     105|   35.51|  29.67|0.21| 0.3| 0.3|  39.61|3.72|
|박혜민|KGC인삼공사|  L|1440.6|   68|      85|    35.6|   28.6|0.13|0.15|0.19|  28.98|2.95|
+------+-----------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
only showing top 5 rows



In [39]:
ibk_rate = spark.sql("""
          select *
          from player_rate
          where team like 'IBK기업은행'
          """)
ibk_rate.show(5)

+------+-----------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|  name|       team|pos| score|error|setcount|at_srate|at_rate|  sv| set|  bl|rs_rate|  dg|
+------+-----------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|신연경|IBK기업은행| Li|1158.6|    3|     112|     0.0|    0.0| 0.0| 0.6| 0.0|  39.43|5.28|
|김수빈|IBK기업은행| Li|  83.6|    0|      26|     0.0|    0.0| 0.0|0.08| 0.0|  26.67| 1.5|
|양유경|IBK기업은행| Li|   1.0|    0|       1|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0| 0.0|
|표승주|IBK기업은행|  L|2175.4|   57|     102|   36.77|  33.03|0.17|0.24|0.27|  26.62|2.96|
|김주향|IBK기업은행|  L|1704.4|   76|      91|   33.01|  26.35|0.09|0.11|0.22|  26.87|2.93|
+------+-----------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
only showing top 5 rows



In [40]:
hguk_rate = spark.sql("""
          select *
          from player_rate
          where team like '흥국생명'
          """)
hguk_rate.show(5)

+------+--------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|  name|    team|pos| score|error|setcount|at_srate|at_rate|  sv| set|  bl|rs_rate|  dg|
+------+--------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|김해란|흥국생명| Li| 596.4|    2|      58|     0.0|    0.0| 0.0|1.02| 0.0|  31.28|5.78|
|도수빈|흥국생명| Li| 589.2|    3|      60|     0.0|    0.0| 0.0|0.63| 0.0|  38.38| 4.9|
|박상미|흥국생명| Li|  42.2|    0|      16|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0|1.44|
|현무린|흥국생명| Li|   1.0|    0|       1|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0| 0.0|
|김미연|흥국생명|  L|2238.2|  102|     111|   33.29|  27.16|0.24|0.32|0.19|  25.57|3.66|
+------+--------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
only showing top 5 rows



In [41]:
pepper_rate = spark.sql("""
          select *
          from player_rate
          where team like '페퍼저축은행'
          """)
pepper_rate.show(5)

+------+------------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|  name|        team|pos| score|error|setcount|at_srate|at_rate|  sv| set|  bl|rs_rate|  dg|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     0.0|    0.0| 0.0| 0.7| 0.0|  32.04| 3.1|
|김세인|페퍼저축은행| Li| 334.4|    0|      84|     0.0|    0.0| 0.0|0.23| 0.0|  30.56| 2.2|
|이은지|페퍼저축은행| Li|   7.8|    0|       4|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0| 0.5|
|이한비|페퍼저축은행|  L|1930.4|   91|     105|   30.09|  25.03| 0.1| 0.2|0.12|  29.35|3.18|
|박경현|페퍼저축은행|  L|1714.8|   94|      96|   35.57|  30.66|0.16|0.23| 0.1|  22.13|2.86|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
only showing top 5 rows



In [44]:
# 선수 데이터 저장 @ 하둡

player_rate2.write.format('csv').save('hdfs://localhost:9000/data/player_rate.csv',header='true')

In [45]:
data = spark.read.csv("hdfs://localhost:9000/data/player_rate.csv", header='true', inferSchema='true')
data.show(5)

+------+------------+---+------+-----+--------+--------+-------+---+----+---+-------+----+
|  name|        team|pos| score|error|setcount|at_srate|at_rate| sv| set| bl|rs_rate|  dg|
+------+------------+---+------+-----+--------+--------+-------+---+----+---+-------+----+
|임명옥|한국도로공사| Li|1475.0|    3|     117|     0.0|    0.0|0.0|1.05|0.0|  55.24| 5.9|
|신연경| IBK기업은행| Li|1158.6|    3|     112|     0.0|    0.0|0.0| 0.6|0.0|  39.43|5.28|
|김연견|    현대건설| Li|1099.0|    4|     113|     0.0|    0.0|0.0|0.79|0.0|   39.2|4.96|
|오지영|    GS칼텍스| Li| 880.6|    2|      94|     0.0|    0.0|0.0|0.51|0.0|  39.37|3.99|
|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     0.0|    0.0|0.0| 0.7|0.0|  32.04| 3.1|
+------+------------+---+------+-----+--------+--------+-------+---+----+---+-------+----+
only showing top 5 rows



In [46]:
# 구단별 데이터 저장 @ 하둡

hgun_rate.write.format('csv').save('hdfs://localhost:9000/data/hgun_rate.csv',header='true')
dogong_rate.write.format('csv').save('hdfs://localhost:9000/data/dogong_rate.csv',header='true')
gscal_rate.write.format('csv').save('hdfs://localhost:9000/data/gscal_rate.csv',header='true')
insam_rate.write.format('csv').save('hdfs://localhost:9000/data/insam_rate.csv',header='true')
ibk_rate.write.format('csv').save('hdfs://localhost:9000/data/ibk_rate.csv',header='true')
hguk_rate.write.format('csv').save('hdfs://localhost:9000/data/hguk_rate.csv',header='true')
pepper_rate.write.format('csv').save('hdfs://localhost:9000/data/pepper_rate.csv',header='true')

In [47]:
# 데이터 잘 저장됐는지 확인

data = spark.read.csv("hdfs://localhost:9000/data/pepper_rate.csv", header='true', inferSchema="true")
data.show(5)

+------+------------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|  name|        team|pos| score|error|setcount|at_srate|at_rate|  sv| set|  bl|rs_rate|  dg|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     0.0|    0.0| 0.0| 0.7| 0.0|  32.04| 3.1|
|김세인|페퍼저축은행| Li| 334.4|    0|      84|     0.0|    0.0| 0.0|0.23| 0.0|  30.56| 2.2|
|이은지|페퍼저축은행| Li|   7.8|    0|       4|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0| 0.5|
|이한비|페퍼저축은행|  L|1930.4|   91|     105|   30.09|  25.03| 0.1| 0.2|0.12|  29.35|3.18|
|박경현|페퍼저축은행|  L|1714.8|   94|      96|   35.57|  30.66|0.16|0.23| 0.1|  22.13|2.86|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
only showing top 5 rows



## 데이터 모델 생성 - 22/